# Neural Networks

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# reading in the data
columns = pd.read_csv('data/features2.txt', header = None)

X_train = pd.read_csv('data/Train/X_train.txt', names = columns[0], sep=' ')
X_test = pd.read_csv('data/Test/X_test.txt', names = columns[0], sep=' ')

y_train = pd.read_csv('data/Train/y_train.txt', sep=' ')
y_test = pd.read_csv('data/Test/y_test.txt', sep=' ')

**PLAN**

For neural networks, you can simply try a fully connected NN with a few layers, 

first ignoring the time component and previous activity

then second, you can do RNNs by feeding in say 10 time-steps and predicting the next activity